---
# Évaluation des modèles de classification : Exemple (un peu) plus réaliste
---

<center><img src="https://python.gel.ulaval.ca/media/sio-u009/mlprocess_4.png" alt="Processus d'apprentissage automatique" width="50%"/></center>

Dans cette séquence nous allons repartir d'un ensemble de données beaucoup plus complexe à base d'images de chiffres manuscrits dont le but est d'identifier le chiffre écrit. C'est un problème multiclasse (10 chiffres de 0 à 9) et entrainer un modèle en se basant sur les différentes métriques vu auparavant ne sera pas aussi simple. 

Cet ensemble de données servira également pour la transition vers l'apprentissage profond.

Importons d'abord les librairies nécessaires.

In [ ]:
import gzip
import struct
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve, validation_curve
from sklearn.datasets import load_digits
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_recall_curve, classification_report, fbeta_score

Et définissons l'ensemble de données, téléchargeons le et chargeons le en mémoire.

In [ ]:
target_names = ['class 0', 'class 1', 'class 2', 'class 3', 
                'class 4', 'class 5', 'class 6', 'class 7', 'class 8', 'class 9']

In [ ]:
def loadData(src, cimg):
    with gzip.open(src) as gz:
        n = struct.unpack('I', gz.read(4))
        # Read magic number.
        if n[0] != 0x3080000:
            raise Exception('Invalid file: unexpected magic number.')
        # Read number of entries.
        n = struct.unpack('>I', gz.read(4))[0]
        if n != cimg:
            raise Exception('Invalid file: expected {0} entries.'.format(cimg))
        crow = struct.unpack('>I', gz.read(4))[0]
        ccol = struct.unpack('>I', gz.read(4))[0]
        if crow != 28 or ccol != 28:
            raise Exception('Invalid file: expected 28 rows/cols per image.')
        # Read data.
        res = np.frombuffer(gz.read(cimg * crow * ccol), dtype = np.uint8)
    return res.reshape((cimg, crow * ccol))

def loadLabels(src, cimg):
    with gzip.open(src) as gz:
        n = struct.unpack('I', gz.read(4))
        # Read magic number.
        if n[0] != 0x1080000:
            raise Exception('Invalid file: unexpected magic number.')
        # Read number of entries.
        n = struct.unpack('>I', gz.read(4))
        if n[0] != cimg:
            raise Exception('Invalid file: expected {0} rows.'.format(cimg))
        # Read labels.
        res = np.frombuffer(gz.read(cimg), dtype = np.uint8)
    return res.reshape((cimg, 1))

In [ ]:
import wget 

!rm './MNIST/raw/train-images-idx3-ubyte.gz' './MNIST/raw/train-labels-idx1-ubyte.gz' './raw/MNIST/t10k-images-idx3-ubyte.gz' './MNIST/raw/t10k-labels-idx1-ubyte.gz'

wget.download('https://github.com/iid-ulaval/EEAA-datasets/raw/master/MNIST/train-images-idx3-ubyte.gz', './MNIST/raw/train-images-idx3-ubyte.gz')
wget.download('https://github.com/iid-ulaval/EEAA-datasets/raw/master/MNIST/train-labels-idx1-ubyte.gz', './MNIST/raw/train-labels-idx1-ubyte.gz')
wget.download('https://github.com/iid-ulaval/EEAA-datasets/raw/master/MNIST/t10k-images-idx3-ubyte.gz', './MNIST/raw/t10k-images-idx3-ubyte.gz')
wget.download('https://github.com/iid-ulaval/EEAA-datasets/raw/master/MNIST/t10k-labels-idx1-ubyte.gz', './MNIST/raw/t10k-labels-idx1-ubyte.gz')


In [ ]:
X_train = loadData('./MNIST/raw/train-images-idx3-ubyte.gz', 60000)
y_train = loadLabels('./MNIST/raw/train-labels-idx1-ubyte.gz', 60000)

Vous pouvez regarder quelques exemples de données avec le code ci-dessous : 

In [ ]:
example_i = 5050
plt.imshow(X_train[example_i,:].reshape(28,28), cmap="gray_r")

## Découpage des données de validation

Ici vous devez définir le découpage train/valid pour pouvoir sélectionner votre modèle. Ne touchez pas au test avant la fin de l'exercice ;-) 

Vu qu'on est en multiclasse, n'oubliez pas d'utiliser la stratification de votre ensemble de données pour garantir que toutes les classes sont bien représentées dans l'ensemble de validation (our cela utilisez le paramètre `stratify` de [`train_test_split()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) en lui passant les étiquettes `y`).

In [ ]:
# X_train, X_val, y_train, y_val = ...

## Courbe d'apprentissage

Choisissez ici un classificateur aisément multiclasse (par exemple un arbre de décision : [`DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)) ou utilisez n'importe quel claissificateur binaire en *OneVsRest*. Le code après va permettre de l'évaluer en validation croisée selon son exactitude.

Nous allons étudier à quel point l'apport de nouveaux exemples aident l'apprentissage en favorisant la généralisation et quelle est la limite de la capacité d'un modèle.

In [ ]:
# from sklearn. ... import ... 

# estimator = ...

In [ ]:
train_sizes=np.linspace(.1, 1.0, 5)
train_sizes, train_scores, valid_scores = learning_curve(
        estimator, X_train, y_train, cv=5, n_jobs=-1, train_sizes=train_sizes, scoring='accuracy', verbose=1)

# Statistiques sur les différents plis pour affichage ensuite
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
valid_scores_mean = np.mean(valid_scores, axis=1)
valid_scores_std = np.std(valid_scores, axis=1)

In [ ]:
plt.figure()
plt.xlabel("Training examples")
plt.ylabel("Score")
plt.grid()

# un ecart type sur les plis
plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.3, color="r")
plt.fill_between(train_sizes, valid_scores_mean - valid_scores_std, valid_scores_mean + valid_scores_std, alpha=0.3, color="g")

plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
plt.plot(train_sizes, valid_scores_mean, 'o-', color="g", label="Cross-validation score")
plt.legend(loc="best")

Cette figure présente que les 15000 premiers exemples sont utiles, mais qu'en ajouter n'aide pas significativement l'apprentissage de par la limite éventuelle de la capacité du modèle ou le bruit aléatorique des données.

## Courbe de validation

Là où la courbe d'apprentissage mesure la performance en fonctiond du nombre d'exemples vus, la courbe de validation permet de mesurer la performance en fonction de la capacité du modèle et des hyperparamètres choisis. Dans l'exemple de code suivant (à compléter selon le modèle que vous avez choisi), il est possible de visualiser l'impach d'un seul hyperparamètre.

Dans le cas de l'arbre de décision par exemple nous pourrions choisir l'hyperparamètre `max_depth` et le faire varier de manière exponentielle entre 1 et 256 : `[1,2,4,8,16,32,64,128]`

In [ ]:
hyper_name = 'max_depth'
param_range = [1,2,4,8,16,32,64,128,256]

# Calcul de la courbe de validation avec validation croisée
train_scores, valid_scores = validation_curve(estimator, X_train, y_train, param_name=hyper_name, param_range=param_range, cv=5, scoring="accuracy", n_jobs=-1, verbose=1)

# Calcul des statistiques pour visualisation
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
valid_scores_mean = np.mean(valid_scores, axis=1)
valid_scores_std = np.std(valid_scores, axis=1)

# Visualisation
plt.figure()
plt.title("Validation Curve")
plt.xlabel("hyperparameter")
plt.ylabel("Score")
plt.ylim(0.0, 1.1)
lw = 2

plt.semilogx(param_range, train_scores_mean, label="Training score", color="darkorange", lw=lw)
plt.fill_between(param_range, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.3, color="darkorange", lw=lw)
plt.semilogx(param_range, valid_scores_mean, label="Cross-validation score", color="navy", lw=lw)
plt.fill_between(param_range, valid_scores_mean - valid_scores_std, valid_scores_mean + valid_scores_std, alpha=0.3, color="navy", lw=lw)
plt.legend(loc="best")
plt.show()

## Entraînement d'un classificateur

Nous avons vu comment choisir le bon nombre de plis selon la courbe d'apprentissage et comment tester les hyperparamètres un par un avec la courbe de validation. À vous maintenant d'essayer d'entrainer le meilleur modèle possible Scikit-Learn sur les données MNIST à l'aide de tous les outils dont vous disposez.

Un exemple sommaire vous est donné ci-dessous.


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(min_samples_split=10, n_jobs=-1, verbose=1)
score = cross_val_score(clf, X_train, y_train, cv=5, verbose=1, n_jobs=-1)
np.mean(score)

## Évaluer le modèle choisi sur le test

Une fois que le classificateur est choisi avec le meilleur ensemble d'hyperparamètres, il est temps de reéentrainer le modèle choisi au complet sur la totalité des données d'entrainement selon ces choix. Ce modèle sera testé sur l'ensemble de test. 

Si vous recommencez le processus après cela parce que vous n'êtes pas satisfait du résultat sur l'ensemble de test, vous commencez à *tricher* et à surapprendre l'ensemble de test. La méthodologie devient bancale, et vous perdez toute garantie de fonctionnement en production (toujours sous l'hypothèse i.i.d  et que la distribution de génération des entrées ne change pas). 

In [ ]:
X_train = loadData('./MNIST/raw/train-images-idx3-ubyte.gz', 60000)
y_train = loadLabels('./MNIST/raw/train-labels-idx1-ubyte.gz', 60000)
X_test = loadData('./MNIST/raw/t10k-images-idx3-ubyte.gz', 10000)
y_test = loadLabels('./MNIST/raw/t10k-labels-idx1-ubyte.gz', 10000)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

N'hésitez pas à partager le résultat obtenu sur le forum en spécifiant le modèle, les hyperparamètres, et la graine de génération aléatoire choisie pour l'entrainement pour la réplicabilité de vos résultats.